<a href="https://colab.research.google.com/github/dnuentsa/cloud-tools-for-analytics/blob/main/notebooks/extract_transform_from_postgres_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose of this notebook
This notebook shows  how to design a simple data extraction pipeline from a PostgreSQL relational database. The goal is to retrieve structured data from multiple tables, process the information into a meaningful format, and store the results in a CSV file for further use in a data pipeline.

 Even if it's a simple one, it shows a a real-world data engineering workflow, where raw data is extracted from relational databases, and transformed into a usable format. The resulting CSV file can be used in ETL (Extract, Transform, Load) pipelines, BI Tools to generate Business dashboards or reports, or machine learning models.

# Prerequisites to run the notebook
- Connect to [https://colab.research.google.com/]() with any Google account.
- if you want to use your student Account, you will get redirected to your usual login page. Enter your student credentials
- Upload this notebook.
- You need to retrieve the credentials to connect to the Postgres database. This is available in the file Datasets/postgres_connection_string.txt on Campus. Open the file and copy the credentials.
- On the left panel, Select the Key/Lock icon to add the connection string to connect to your postgres instance.  Adding a secret here instead of inserting it in your python notebook code will prevent you to share your credentials to anyone having access to your source code. or commit your credentials to online repositories such as github.
- Select "Add new secret"
- Use the name postgres_connection_string
- Paste the connection string in the "Value" Column.
- Make sure to toggle ON the column "Notebook access"

# Database Schema
Review the list of available tables in the database and the relationship between them.
[Database Schema](https://github.com/dnuentsa/cloud-tools-for-analytics/blob/main/resources/postgresql/database_schema.png)

# Main steps
Main steps in this notebook include:
## Connectin to the Database
Establish a secure connection to a remote PostgreSQL database using Python’s psycopg2 library.
Credentials and connection details are retrieved from your secrets defined above.

## Preview the tables
Retrieve the list of available tables to understand the database schema.
Preview a few rows from each table to get a sense of the data.

## Query and Transform Data
Extract relevant records from the trips table.
Use SQL JOINs to replace foreign key IDs with actual values (e.g., customer names, car details, city names).
Convert the data into a structured Pandas DataFrame.

## Store the Processed Data
Export the cleaned and structured dataset to a CSV file.
This file can be used for further data analysis, machine learning, or reporting.

In [ ]:
pip install psycopg2 pandas sqlalchemy

In [ ]:
import zipfile
import os
import io
import psycopg2
import requests

# Function to download zip file from a remote location
def download_zip(url, save_path):
    response = requests.get(url)
    with open(save_path, 'wb') as f:
        f.write(response.content)



# Establish a connection to the database
conn = psycopg2.connect(connection_string)

# Create a cursor object to execute SQL queries
cur = conn.cursor()


In [23]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# Retrieve the postgres connection string from secrets
from google.colab import userdata
connection_string = userdata.get('postgres_connection_string')

# Establish Connection to PostgreSQL
try:
    engine = create_engine(connection_string)
    print("Connected to the database successfully!")
except Exception as e:
    print("Failed to connect to the database:", e)

✅ Connected to the database successfully!


In [ ]:
# List Tables in the Database
query_list_tables = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
"""
# Read Some Lines from Each Table
tables = pd.read_sql(query_list_tables, engine)
print("📋 Tables in the database:\n", tables)
table_names = tables["table_name"].tolist()
for table in table_names:
    query = f"SELECT * FROM {table} LIMIT 5;"
    print(f"\n🔹 Preview of table: {table}")
    print(pd.read_sql(query, engine))

In [ ]:
# Read Some Lines from Each Table
table_names = tables["table_name"].tolist()
for table in table_names:
    query = f"SELECT * FROM {table} LIMIT 5;"
    print(f"\n🔹 Preview of table: {table}")
    print(pd.read_sql(query, conn))

In [ ]:
# Read Entire Trips Table with Joins (Using SQLAlchemy)
query_trips = """
SELECT
    trips.id, trips.pickup_time, trips.dropoff_time, trips.distance, trips.revenue,
    cities.city_name AS trip_city,
    customers.name AS customer_name, customers.email,
    cars.brand AS car_brand, cars.model AS car_model, cars.year
FROM trips
JOIN cars ON trips.car_id = cars.id
JOIN customers ON trips.customer_id = customers.id
JOIN cities ON cars.city_id = cities.city_id;
"""

df_trips = pd.read_sql(query_trips, engine)

# Preview the DataFrame
print("\nPreview of Trips DataFrame:")
print(df_trips.head())

In [ ]:
# Homework Generate some visualizations here from the data in df_trips dataframe.
# e.g Revenue per Car Model, Trips per city, ...


In [ ]:
# Homework Get the average trips rating by city in a dataframe
query_rating = """
WRITE THE QUERY TO GET THE AVERAGE RATING PER CITY HERE
"""

df_rating = pd.read_sql(query_rating, engine)

# Preview the DataFrame
print("\n📊 Preview of Ratings DataFrame:")
# TODO PREVIEW YOUR DATAFRAME HERE

In [ ]:
#

In [ ]:
# Export to CSV and generate a zip file
import zipfile
import os


df_trips.to_csv("trips_data.csv", index=False)
# TODO Export your ratings per city in a csv file named rating_data.csv


# Create a zip file
def zip_file(file_path, zip_file_name):
    with zipfile.ZipFile(zip_file_name, 'a', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(file_path)
    print(f"\n'{file_path}' zipped to '{zip_file_name}' successfully!")

# Delete previous zip file
zip_file_name = "exported_data.zip"
if os.path.exists(zip_file_name):
    os.remove(zip_file_name)
    print(f"File '{zip_file_name}' deleted successfully.")

zip_file("trips_data.csv", zip_file_name)
zip_file("rating_data.csv", zip_file_name)

print("\nData exported to 'trips_data.csv' and trips_data.zip successfully!")

# Download the files to your computer
from google.colab import files
files.download('trips_data.zip')

In [ ]:
# Free Upp the resources
engine.dispose()